Modeling: Pixelized
===================

This script fits multiple multi-wavelength `Imaging` datasets of a galaxy with a model where:

 - The galaxy's light is modeled using an `Inversion` with a rectangular pixelization and constant regularization
 scheme.

Two images are fitted, corresponding to a greener ('g' band) and redder image (`r` band).

This is an advanced script and assumes previous knowledge of the core **PyAutoGalaxy** API for galaxy modeling. Thus,
certain parts of code are not documented to ensure the script is concise.

In [ ]:

%matplotlib inline
from pyprojroot import here
workspace_path = str(here())
%cd $workspace_path
print(f"Working Directory has been set to `{workspace_path}`")

from pathlib import Path
import autofit as af
import autogalaxy as ag
import autogalaxy.plot as aplt

__Colors__

The colors of the multi-wavelength image, which in this case are green (g-band) and red (r-band).

The strings are used for load each dataset.

In [ ]:
color_list = ["g", "r"]

__Pixel Scales__

Every multi-wavelength dataset can have its own unique pixel-scale.

In [ ]:
pixel_scales_list = [0.08, 0.12]

__Dataset__

Load and plot each multi-wavelength galaxy dataset, using a list of their waveband colors.

Note how the disk appears brighter in the g-band image, whereas the bulge is clearer in the r-band image.
Multi-wavelength image can therefore better decompose the structure of galaxies.

In [ ]:
dataset_type = "multi"
dataset_label = "imaging"
dataset_name = "simple"

dataset_path = Path("dataset") / dataset_type / dataset_label / dataset_name

dataset_list = [
    ag.Imaging.from_fits(
        data_path=Path(dataset_path) / f"{color}_data.fits",
        psf_path=Path(dataset_path) / f"{color}_psf.fits",
        noise_map_path=Path(dataset_path) / f"{color}_noise_map.fits",
        pixel_scales=pixel_scales,
    )
    for color, pixel_scales in zip(color_list, pixel_scales_list)
]

for dataset in dataset_list:
    dataset_plotter = aplt.ImagingPlotter(dataset=dataset)
    dataset_plotter.subplot_dataset()

__Mask__

The model-fit requires a 2D mask defining the regions of the image we fit the galaxy model to the data, which we define
and use to set up the `Imaging` object that the galaxy model fits.

For multi-wavelength galaxy modeling, we use the same mask for every dataset whenever possible. This is not
absolutely necessary, but provides a more reliable analysis.

In [ ]:
mask_list = [
    ag.Mask2D.circular(
        shape_native=dataset.shape_native, pixel_scales=dataset.pixel_scales, radius=3.0
    )
    for dataset in dataset_list
]


dataset_list = [
    dataset.apply_mask(mask=mask) for imaging, mask in zip(dataset_list, mask_list)
]

for dataset in dataset_list:
    dataset_plotter = aplt.ImagingPlotter(dataset=dataset)
    dataset_plotter.subplot_dataset()


__JAX & Preloads__

The `autolens_workspace/*/imaging/features/pixelization/modeling` example describes how JAX required preloads in
advance so it knows the shape of arrays it must compile functions for.

In [ ]:
mesh_shape = (20, 20)
total_mapper_pixels = mesh_shape[0] * mesh_shape[1]

total_linear_light_profiles = 0

preloads = ag.Preloads(
    mapper_indices=ag.mapper_indices_from(
        total_linear_light_profiles=total_linear_light_profiles,
        total_mapper_pixels=total_mapper_pixels,
    ),
    source_pixel_zeroed_indices=ag.util.mesh.rectangular_edge_pixel_list_from(
        total_linear_light_profiles=total_linear_light_profiles,
        shape_native=mesh_shape,
    ),
)

__Analysis__

We create an `Analysis` object for every dataset.

In [ ]:
analysis_list = [
    ag.AnalysisImaging(dataset=dataset, preloads=preloads, use_jax=True)
    for dataset in dataset_list
]

__Model__

We compose our galaxy model using `Model` objects, which represent the galaxies we fit to our data. In this 
example we fit a galaxy model where:

 - The galaxy's light uses a `RectangularMagnification` meshwhose resolution is free to vary [2 parameters]. 

 - This pixelization is regularized using a `Constant` scheme which smooths every pixel 
 equally, where its `regularization_coefficient` varies across the datasets [2 parameter]. 

The number of free parameters and therefore the dimensionality of non-linear parameter space is N=4.

In [ ]:
pixelization = af.Model(
    ag.Pixelization,
    mesh=af.Model(ag.mesh.RectangularMagnification, shape=mesh_shape),
    regularization=ag.reg.Constant,
)

galaxy = af.Model(ag.Galaxy, redshift=0.5, pixelization=pixelization)

model = af.Collection(galaxies=af.Collection(galaxy=galaxy))

We now combine them using the factor analysis class, which allows us to fit the two datasets simultaneously.

We make the regularization coefficient a free parameter across every analysis object, ensuring different levels
of regularization are applied to each wavelength of data.

In [ ]:
analysis_factor_list = []

for analysis in analysis_list:

    model_analysis = model.copy()
    model_analysis.galaxies.galaxy.pixelization.regularization.coefficient = (
        af.LogUniformPrior(lower_limit=1e-4, upper_limit=1e4)
    )

    analysis_factor = af.AnalysisFactor(prior_model=model_analysis, analysis=analysis)

    analysis_factor_list.append(analysis_factor)

factor_graph = af.FactorGraphModel(*analysis_factor_list, use_jax=True)

The `info` of the model shows us there are two models each with their own regularization coefficient as a free parameter.

In [ ]:
print(factor_graph.global_prior_model.info)

__Search__

The model is fitted to the data using a non-linear search. In this example, we use the nested sampling algorithm 
Nautilus (https://nautilus.readthedocs.io/en/latest/).

A full description of the settings below is given in the beginner modeling scripts, if anything is unclear.

In [ ]:
search = af.Nautilus(
    path_prefix=Path("multi") / "features",
    name="pixelized",
    unique_tag=dataset_name,
    n_live=100,
    n_batch=50,  # GPU lens model fits are batched and run simultaneously, see VRAM section below.
)

__Model-Fit__

In [ ]:
result_list = search.fit(model=factor_graph.global_prior_model, analysis=factor_graph)

__Result__

The result object returned by this model-fit is a list of `Result` objects, because we used a factor graph.
Each result corresponds to each analysis, and therefore corresponds to the model-fit at that wavelength.

For example, close inspection of the `max_log_likelihood_instance` of the two results shows that all parameters,
except the `coefficient` of the galaxy's pixelization identical.

In [ ]:
print(result_list[0].max_log_likelihood_instance)
print(result_list[1].max_log_likelihood_instance)

Plotting each result's galaxies shows that the source appears different, owning to its different intensities.

In [ ]:
for result in result_list:
    galaxies_plotter = aplt.GalaxiesPlotter(
        galaxies=result.max_log_likelihood_galaxies, grid=result.grids.lp
    )
    galaxies_plotter.subplot_galaxies()

    fit_plotter = aplt.FitImagingPlotter(fit=result.max_log_likelihood_fit)
    fit_plotter.subplot_fit()

The `Samples` object still has the dimensions of the overall non-linear search (in this case N=15). 

Therefore, the samples is identical in every result object.

In [ ]:
for result in result_list:
    plotter = aplt.NestPlotter(samples=result.samples)
    plotter.corner_cornerpy()

Checkout `autogalaxy_workspace/*/results` for a full description of analysing results in **PyAutoGalaxy**.